In [ ]:
###########################################################################################################
## 雲検出サンプル
##########################################################################################################
import sys
import os
import logging
import numpy as np
import leveldb
import caffe
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

cloud_color = (0,0,255) # red
sky_color = (0,255,255) # blue
bg_color = (0,255,0) # blue

#block_size = 18
block_size = 28
#block_size = 80
#block_size = 64
#block_size = 60
#block_size = 100

count = 0
cloud = 0
sky = 0
ground = 0
cloudarea = 0

input_dir="input/"
output_dir="output/"


#model_dir="20171013-144912-b3bf_epoch_20.0/"
#model_name="snapshot_iter_5220.caffemodel"

#model_dir="20171101-132438-0eb2_epoch_20.0/_A"
#model_name="snapshot_iter_2620.caffemodel"

#model_dir="20171221-122257-d626_epoch_25.0/"
#model_name="snapshot_iter_8700.caffemodel"

#model_dir="20180402-173959-29ca_epoch_30.0/"
#model_name="snapshot_iter_10440.caffemodel"

#model_dir="20180410-130204-1efb_epoch_30.0/"
#model_name="snapshot_iter_10440.caffemodel"
#four conv

#model_dir="20180623-093758-568b_epoch_20.0/"
#model_name="snapshot_iter_6000.caffemodel"
#64_28_layer2_200_500_1300 0.005 conv2

model_dir="20180628-053248-3e16_epoch_20.0/"
model_name="snapshot_iter_6000.caffemodel"
#64_64_210_310_910_conv2

#model_dir="20180506-105828-7518_epoch_30.0/"
#model_name="snapshot_iter_10440.caffemodel"
#28_28



input_file = "data1.jpeg"
#input_file = "data2.jpg"
#input_file = "data3.jpg"
#input_file = "data14.jpeg"
#input_file = "data17.jpeg"
#input_file = "data18.jpeg"
#input_file = "data19.jpeg"
#input_file = "data20.jpeg"
#input_file = "data21.jpeg"
#input_file = "data22.jpeg"
#input_file = "data23.jpeg"
#input_file = "data24.JPG"
#input_file = "data25.JPG"
#input_file = "data26.jpg"
#input_file = "data27.JPG"
#input_file = "data28.JPG"


output_file = input_file

import time


def detection(img, cnn, mean):
    transformer = caffe.io.Transformer({'data': cnn.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))  # 色チャネルを最初の次元に動かす
    transformer.set_raw_scale('data', 255)      # スケールを[0, 1]から[0, 255]へ
    transformer.set_channel_swap('data', (2,1,0))  # RGBからBGRへ
    transformed_image = transformer.preprocess('data', img)
    transformed_image = transformed_image - mean
    
    cnn.blobs['data'].data[...] = transformed_image
    
    output = cnn.forward()
    result = output['softmax']
    return result

if __name__ == '__main__':
    caffe.set_mode_gpu()
       
    # CNNネットワークモデルの読み込み
    prototxt = model_dir + 'deploy.prototxt' 
    caffemodel = model_dir + model_name
    cnn = caffe.Classifier(prototxt, caffemodel)

    # ネットワークの各レイヤーのサイズを出力
    print "--- Layer Parameter -----------------------------------"
    for layername, params in cnn.params.items():
        print layername, params[0].data.shape

    # ネットワークの各ブロブのサイズを出力
    print "--- Blob Infomation -----------------------------------"
    for blobname, blobs in cnn.blobs.items():
        print blobname, blobs.data.shape

    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( model_dir + "mean.binaryproto" , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    mean = arr[0,:,:,:]
    mean = mean[:,0:227,0:227]

    img = caffe.io.load_image(input_dir + input_file, color = True)
    rimg = cv2.imread(input_dir + input_file)
       
    nW = int(img.shape[1]/block_size);
    nH = int(img.shape[0]/block_size);
    print "image size : ", img.shape
    print "blocks : ", nW, "x", nH 
    nW1 = int(img.shape[1]/block_size);
    nH1 = int(img.shape[0]/block_size);
    
    fa = open('result_annotation/data1.txt', 'w')
    #fa = open('result_annotation/data2.txt', 'w')
    #fa = open('result_annotation/data3.txt', 'w')
    #fa = open('result_annotation/data14.txt', 'w')
    #fa = open('result_annotation/data17.txt', 'w')
    
    
    start = time.time()
    c2 = 0
    for h in range(nH):
        for w in range(nW):
            roi=img[h*block_size:(h+1)*block_size,w*block_size:(w+1)*block_size]
            
            result = detection(roi,cnn,mean)
            label = np.argmax(result)
            if label == 1:
                cv2.circle(rimg, (w*block_size+block_size/2, h*block_size+block_size/2), block_size/2, sky_color, 3)
               
            if label == 0:
                cv2.circle(rimg, (w*block_size+block_size/2, h*block_size+block_size/2), block_size/2, cloud_color, 3)
               
            if label == 2:
                cv2.circle(rimg, (w*block_size+block_size/2, h*block_size+block_size/2), block_size/2, bg_color, 3)
                
            fa.write("0 %d %d %d %d %d\n" % (label, h*block_size+block_size/2, w*block_size+block_size/2, block_size, block_size))

    fa.close()
                
    elapsed_time = time.time() - start
    cv2.imshow("result", rimg[0:block_size*nH, 0:block_size*nW])
    
    cv2.imwrite(output_dir + output_file, rimg[0:block_size*nH, 0:block_size*nW])
    
    
    
    
    
    
    
    
a = 0
b = 0
c = 0
d = 0
e = 0
count1 = 0
cloud1 = 0
sky1 = 0
ground1 = 0

fc = open('result_error/data1.txt' , 'w')
for line in open("result_annotation/data1.txt", "r"):
    data = line.split()
    a = data[2]
    data[2] = data[3]
    data[3] = a
    d = int(data[1])
    if d==2:
        e=1
        sky+=1
        
    if d==1:
        e=2
        ground+=1
        
    if d==0:
        e=0
        cloud+=1
    for line1 in open('/home/lab/tokutake/project/BlockwiseLabeling/build/data1/annotation.txt', 'r'):
        fb = line1.split()
        if int(fb[1])==0:
            cloud1 += 1
        if int(fb[1])==1:
            sky1 += 1
        if int(fb[1])==2:
            ground1 += 1
        if int(data[2]) == int(fb[2])+14 and int(data[3]) == int(fb[3])+14:
            if e != int(fb[1]):
                b = int(fb[2])
                c = int(fb[3])
                count1 += 1
                fc.write('0 4 %d %d %d %d\n' % (b,c,block_size,block_size))
                
fc.close()



count = cloud+sky
sum1 = count+ground
g = ((float)(sum1-count1)/sum1) * 100
h = (cloud1/(float)(sky1+cloud1)) * 100  
cloudarea = ((float)(cloud) /(float)(count)) * 100


f = open('result_data/data1.txt', 'w')
#f = open('result_data/data2.txt', 'w')
#f = open('result_data/data3.txt', 'w')
#f = open('result_data/data14.txt', 'w')
#f = open('result_data/data17.txt', 'w')

f.write('cloud=%d\n' %cloud)
f.write('sky=%d\n' %sky)
f.write('ground=%d\n' %ground)
f.write('sum=%d\n\n'%sum1)
f.write('error=%d\n\n'%count1)
f.write('correct cloud area = %f\n' %h)
f.write('cloud area = %f\n\n' %cloudarea)
f.write('error of cloud area = %f\n\n' %abs(h-cloudarea))
f.write('time = %f\n\n' %elapsed_time)
f.write('all accuracy = %f\n' %g)

f.close() 
   
    
cv2.waitKey(0)
cv2.destroyAllWindows()    



/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


--- Layer Parameter -----------------------------------
conv1 (210, 3, 5, 5)
conv12 (210, 210, 5, 5)
conv2 (310, 210, 5, 5)
conv21 (310, 310, 5, 5)
ip1 (910, 310)
ip12 (910, 910)
ip2 (3, 910)
--- Blob Infomation -----------------------------------
data (1, 3, 64, 64)
scaled (1, 3, 64, 64)
conv1 (1, 210, 60, 60)
pool1 (1, 210, 30, 30)
conv12 (1, 210, 26, 26)
pool12 (1, 210, 13, 13)
conv2 (1, 310, 9, 9)
pool13 (1, 310, 5, 5)
conv21 (1, 310, 1, 1)
pool2 (1, 310, 1, 1)
ip1 (1, 910)
ip12 (1, 910)
ip2 (1, 3)
softmax (1, 3)
image size :  (960, 1280, 3)
blocks :  45 x 34
